In [71]:
import sys

sys.path.append('../')
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

from dotenv import load_dotenv
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
from matplotlib import pyplot
import re
from textblob import TextBlob
import random

load_dotenv()

True

In [72]:
sp = spotipy.Spotify(
    auth_manager=SpotifyClientCredentials(
        client_id=os.getenv("SPOTIFY_CLIENT_ID"),
        client_secret=os.getenv("SPOTIFY_API_KEY")
    ))

Functions:

In [73]:
def df_cleaner(df):
    df.drop_duplicates('name', inplace=True)
    df = df.drop(
        [
            'album',
            'artists',
            'available_markets',
            'disc_number',
            'explicit',
            'external_ids',
            'external_urls',
            'href',
            'id',
            'is_local',
            'episode',
            'track',
            '...',
            'track_href',
            'analysis_url',
            'uri',
            'preview_url',
            'type',
            'name',
            'preview_url',
        ],
        errors='ignore',
        axis='columns'
    )
    df = df.dropna(axis = 0, how = 'all')
    df = df.reindex(sorted(df.columns), axis=1)

    return df


def get_top_tracks(max_page, per_page, keyword):
    current_offset = 1
    final_data = []
    results = sp.search(q='top', limit=per_page, type="track")
    final_results = results['tracks']['items']

    while current_offset <= max_page and results['tracks'].get('next', ''):
        current_offset += 1
        results = sp.search(q=keyword, offset=current_offset)
        final_results.extend(results['tracks']['items'])

    for i in final_results:
        audio_features = sp.audio_features(i["uri"])
        # print(audio_features)
        final_data.append({**i, **audio_features[0]})

    return final_data


def index_to_instance(df, index=None):
    if index:
        return XYZ(df)[index][1]
    else:
        return XYZ(df)


def XYZ(df):
    return sorted(list(zip(list(df.index.codes[0].data), list(df.index.levels[0].array))))


def value_to_index_map(array):
    array1 = zip(array, range(len(array)))
    return array1


class RecSysContentBased():
    def __init__(self):
        pass

    def fit(self, train):
        self.train_set = train
        df1 = cosine_similarity(train)
        self.similarity = df1
        self.distances = pairwise_distances(train, metric='euclidean')

    def evaluate(self, user):
        d = sorted(value_to_index_map(self.distances[user]))
        return list(index_to_instance(self.train_set, d[i][1]) for i in range(len(d)))

    def predict(self):
        pass

    def test(self, testset):
        pass




import and clean the data

In [74]:
baseDf = pd.read_csv('spotify_data.csv')

baseDf.index = [baseDf["name"]]
data = baseDf[[
    'energy', 'danceability', 'loudness', 'liveness', 'valence',
    'duration_ms', 'acousticness', 'speechiness', 'popularity']]
df = df_cleaner(baseDf)
baseDf

,album,artists,available_markets,disc_number,duration_ms,episode,explicit,external_ids,external_urls,href,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_href,analysis_url,time_signature
name,,,,,,,,,,,,,,,,,,,,,
As It Was,"{'album_type': 'single', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,"['AD', 'AE', 'AG', 'AL', 'AM', 'AO', 'AR', 'AT...",1,167303,False,False,{'isrc': 'USSM12200612'},{'spotify': 'https://open.spotify.com/track/4L...,https://api.spotify.com/v1/tracks/4LRPiXqCikLl...,...,0,0.0557,0.342000,0.001010,0.3110,0.662,173.930,https://api.spotify.com/v1/tracks/4LRPiXqCikLl...,https://api.spotify.com/v1/audio-analysis/4LRP...,4
Running Up That Hill (A Deal With God) - 2018 Remaster,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"['AD', 'AE', 'AG', 'AL', 'AM', 'AO', 'AR', 'AT...",1,300840,False,False,{'isrc': 'GBAYE1800884'},{'spotify': 'https://open.spotify.com/track/29...,https://api.spotify.com/v1/tracks/29d0nY7TzCoi...,...,0,0.0596,0.659000,0.002660,0.0546,0.139,108.296,https://api.spotify.com/v1/tracks/29d0nY7TzCoi...,https://api.spotify.com/v1/audio-analysis/29d0...,4
Left and Right (Feat. Jung Kook of BTS),"{'album_type': 'single', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,"['AD', 'AE', 'AG', 'AL', 'AM', 'AO', 'AR', 'AT...",1,154487,False,False,{'isrc': 'USAT22205563'},{'spotify': 'https://open.spotify.com/track/0m...,https://api.spotify.com/v1/tracks/0mBP9X2gPCua...,...,1,0.0324,0.619000,0.000013,0.0901,0.719,101.058,https://api.spotify.com/v1/tracks/0mBP9X2gPCua...,https://api.spotify.com/v1/audio-analysis/0mBP...,4
About Damn Time,"{'album_type': 'single', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,"['AD', 'AE', 'AG', 'AL', 'AM', 'AO', 'AR', 'AT...",1,191822,False,True,{'isrc': 'USAT22202139'},{'spotify': 'https://open.spotify.com/track/1P...,https://api.spotify.com/v1/tracks/1PckUlxKqWQs...,...,0,0.0656,0.099500,0.000000,0.3350,0.722,108.966,https://api.spotify.com/v1/tracks/1PckUlxKqWQs...,https://api.spotify.com/v1/audio-analysis/1Pck...,4
I Like You (A Happier Song) (with Doja Cat),"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"['AD', 'AE', 'AG', 'AL', 'AM', 'AO', 'AR', 'AT...",1,192841,False,True,{'isrc': 'USUM72208023'},{'spotify': 'https://open.spotify.com/track/0O...,https://api.spotify.com/v1/tracks/0O6u0VJ46W86...,...,1,0.0751,0.121000,0.000000,0.1210,0.472,100.964,https://api.spotify.com/v1/tracks/0O6u0VJ46W86...,https://api.spotify.com/v1/audio-analysis/0O6u...,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Bad Boy for Life,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"['AD', 'AE', 'AG', 'AL', 'AM', 'AO', 'AR', 'AT...",1,253067,False,True,{'isrc': 'USBB40580756'},{'spotify': 'https://open.spotify.com/track/2e...,https://api.spotify.com/v1/tracks/2eOuL8KesslT...,...,1,0.4900,0.179000,0.000000,0.2410,0.610,118.934,https://api.spotify.com/v1/tracks/2eOuL8KesslT...,https://api.spotify.com/v1/audio-analysis/2eOu...,4
No Other Love,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"['AD', 'AE', 'AG', 'AL', 'AM', 'AO', 'AR', 'AT...",1,266240,False,False,{'isrc': 'USBB40580604'},{'spotify': 'https://open.spotify.com/track/6p...,https://api.spotify.com/v1/tracks/6p8RmxHGU1WL...,...,0,0.0472,0.318000,0.000000,0.0619,0.711,70.352,https://api.spotify.com/v1/tracks/6p8RmxHGU1WL...,https://api.spotify.com/v1/audio-analysis/6p8R...,4
Five-O (feat. Wyclef & P. Diddy),"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"['AD', 'AE', 'AG', 'AL', 'AM', 'AO', 'AT', 'AU...",1,279773,False,False,{'isrc': 'USBB40707174'},{'spotify': 'https://open.spotify.com/track/3c...,https://api.sp

scale down the data

In [75]:
X = pd.DataFrame(df, columns=list(df.columns))
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

split and cluster the data to 3 cluster

In [76]:
kmeans = KMeans(n_clusters=6, random_state=1234)
kmeans.fit(X_scaled_df)

KMeans(n_clusters=6, random_state=1234)

In [77]:
labels = kmeans.predict(X_scaled_df)
clusters = np.unique(labels)

In [78]:
clusters = kmeans.predict(X_scaled_df)
X["cluster"] = clusters

In [79]:
kmeans2 = KMeans(n_clusters=50,
                 init="k-means++",
                 n_init=50,  # try with 1, 4, 8, 20, 30, 100...
                 max_iter=10,
                 tol=0,
                 algorithm="elkan",
                 random_state=1234)
kmeans2.fit(X_scaled_df)

KMeans(algorithm='elkan', max_iter=10, n_clusters=50, n_init=50,
       random_state=1234, tol=0)

In [80]:
K = range(2, 21)
inertia = []

for k in K:
    kmeans = KMeans(n_clusters=3,
                    random_state=1234)
    kmeans.fit(X_scaled_df)
    inertia.append(kmeans.inertia_)

In [81]:
model = RecSysContentBased()
model.fit(data)

In [82]:
user_input = input('what do you like?')
textBlb = TextBlob(user_input)  # Making our first textblob
textCorrected = str(textBlb.correct())
search_result = baseDf[baseDf['name'].str.contains(user_input, case=False)][:4]
guess_message = ''
search_result

,album,artists,available_markets,disc_number,duration_ms,episode,explicit,external_ids,external_urls,href,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_href,analysis_url,time_signature
name,,,,,,,,,,,,,,,,,,,,,
Cold Heart - PNAU Remix,"{'album_type': 'single', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,"['AD', 'AE', 'AG', 'AL', 'AM', 'AO', 'AR', 'AT...",1,202735,False,False,{'isrc': 'GBUM72104705'},{'spotify': 'https://open.spotify.com/track/6z...,https://api.spotify.com/v1/tracks/6zSpb8dQRaw0...,...,1,0.0317,0.034,0.000042,0.0952,0.942,116.032,https://api.spotify.com/v1/tracks/6zSpb8dQRaw0...,https://api.spotify.com/v1/audio-analysis/6zSp...,4
Ecstasy - Mixed,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"['AD', 'AE', 'AG', 'AL', 'AM', 'AO', 'AR', 'AT...",1,139558,False,False,{'isrc': 'FRU702000211'},{'spotify': 'https://open.spotify.com/track/2P...,https://api.spotify.com/v1/tracks/2PspwQLfDzLU...,...,0,0.0663,0.206,0.037400,0.1430,0.507,119.953,https://api.spotify.com/v1/tracks/2PspwQLfDzLU...,https://api.spotify.com/v1/audio-analysis/2Psp...,4
Mexican Holiday,"{'album_type': 'single', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,"['AD', 'AE', 'AG', 'AL', 'AM', 'AO', 'AR', 'AT...",1,202914,False,False,{'isrc': 'FRU702000298'},{'spotify': 'https://open.spotify.com/track/6E...,https://api.spotify.com/v1/tracks/6E8XDoU8B9Mb...,...,1,0.0268,0.443,0.021300,0.3930,0.953,109.980,https://api.spotify.com/v1/tracks/6E8XDoU8B9Mb...,https://api.spotify.com/v1/audio-analysis/6E8X...,4
Weekend - Jai Wolf Remix,"{'album_type': 'single', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,"['AD', 'AE', 'AG', 'AL', 'AM', 'AO', 'AR', 'AT...",1,249161,False,False,{'isrc': 'FRU701400166'},{'spotify': 'https://open.spotify.com/track/6G...,https://api.spotify.com/v1/tracks/6GXSZaHXyN3Q...,...,1,0.0473,0.279,0.000040,0.1100,0.593,140.049,https://api.spotify.com/v1/tracks/6GXSZaHXyN3Q...,https://api.spotify.com/v1/audio-analysis/6GXS...,4


In [83]:
options = list(enumerate(search_result['name']))
print(*options, sep='\n')

(0, 'Cold Heart - PNAU Remix')
(1, 'Ecstasy - Mixed')
(2, 'Mexican Holiday')
(3, 'Weekend - Jai Wolf Remix')


In [84]:
user_choice_input_data = input(guess_message)
user_choice_id = search_result.iloc[[user_choice_input_data]]['id'].values[0]
user_choice_id

'2PspwQLfDzLUOyaxQ7de5L'

API call for song data from spotify

In [85]:
track_data = sp.track(track_id=user_choice_id)
track_data_featured = {**track_data, **sp.audio_features(track_data['uri'])[0]}
#track_data_featured = sp.audio_features(track_data['uri'])[0]
track_data_featured

{'album': {'album_type': 'album',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1mwP5J1kB8clYx0SoNL0uh'},
    'href': 'https://api.spotify.com/v1/artists/1mwP5J1kB8clYx0SoNL0uh',
    'id': '1mwP5J1kB8clYx0SoNL0uh',
    'name': 'Fabich',
    'type': 'artist',
    'uri': 'spotify:artist:1mwP5J1kB8clYx0SoNL0uh'}],
  'available_markets': ['AD',
   'AE',
   'AG',
   'AL',
   'AM',
   'AO',
   'AR',
   'AT',
   'AU',
   'AZ',
   'BA',
   'BB',
   'BD',
   'BE',
   'BF',
   'BG',
   'BH',
   'BI',
   'BJ',
   'BN',
   'BO',
   'BR',
   'BS',
   'BT',
   'BW',
   'BY',
   'BZ',
   'CA',
   'CD',
   'CG',
   'CH',
   'CI',
   'CL',
   'CM',
   'CO',
   'CR',
   'CV',
   'CW',
   'CY',
   'CZ',
   'DE',
   'DJ',
   'DK',
   'DM',
   'DO',
   'DZ',
   'EC',
   'EE',
   'EG',
   'ES',
   'FI',
   'FJ',
   'FM',
   'FR',
   'GA',
   'GB',
   'GD',
   'GE',
   'GH',
   'GM',
   'GN',
   'GQ',
   'GR',
   'GT',
   'GW',
   'GY',
   'HK',
   'HN',
   'HR',
   'HT',
   'H

In [86]:
track_data_X = pd.DataFrame(track_data_featured, columns=list(df.columns), index=[0])
track_data_scaler = StandardScaler()
scaler.transform(track_data_X)
track_data_X_scaled = scaler.transform(track_data_X)
track_data_X_scaled_df = pd.DataFrame(track_data_X_scaled, columns=track_data_X.columns)
track_data_X_scaled

array([[ 0.30326016,  0.27255468, -0.54777573, -0.11970539, -0.53373005,
        -0.39461789, -0.27108544, -1.33317477, -0.98215205,  0.39636917,
        -0.40675149, -0.26604228,  0.12971834,  2.79522888,  0.36321058]])

predict

In [87]:
track_data_labels = kmeans.predict(track_data_X_scaled_df)
track_data_cluster = np.unique(track_data_labels)
track_data_cluster
track_data_cluster = kmeans.predict(track_data_X_scaled_df)
track_data_X["cluster"] = track_data_cluster
track_data_X_scaled_df

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,time_signature,track_number,valence
0,0.30326,0.272555,-0.547776,-0.119705,-0.53373,-0.394618,-0.271085,-1.333175,-0.982152,0.396369,-0.406751,-0.266042,0.129718,2.795229,0.363211


In [88]:
# random.choice(baseDf[baseDf['cluster'] == 1])
#
#
# random_track = pd.DataFrame(pandas.np.random.random(100))
#
# df_elements = baseDf[baseDf['cluster'] == 1].sample(n=1)

recommend_track = baseDf[X['cluster'] == int(track_data_cluster)].sample()
recommend_track

,album,artists,available_markets,disc_number,duration_ms,episode,explicit,external_ids,external_urls,href,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_href,analysis_url,time_signature
name,,,,,,,,,,,,,,,,,,,,,
Cheat Code,"{'album_type': 'single', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,"['AD', 'AE', 'AG', 'AL', 'AM', 'AO', 'AR', 'AT...",1,170397,False,False,{'isrc': 'CA6D22200362'},{'spotify': 'https://open.spotify.com/track/7J...,https://api.spotify.com/v1/tracks/7JJnlfOlh7dm...,...,1,0.419,0.00268,0.247,0.38,0.439,149.89,https://api.spotify.com/v1/tracks/7JJnlfOlh7dm...,https://api.spotify.com/v1/audio-analysis/7JJn...,4


In [89]:

from IPython.display import IFrame

#track_id = "1rfORa9iYmocEsnnZGMVC4"
IFrame(src=recommend_track['href'].values[0],
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )